In [1]:
from langchain_community.llms import LlamaCpp

import abc


In [2]:

class Model(abc.ABC):
    @abc.abstractmethod
    def send_request(self):
        raise NotImplementedError()

class LlamaCPP(Model):
    def __init__(self, model_path="models/llama-13b-hf_q8_0.gguf", n_gpu_layers=41, n_batch=1024, n_ctx=2048) -> None:
        self.model_path = model_path
        self.n_gpu_layers = n_gpu_layers
        self.n_batch = n_batch
        self.n_ctx = n_ctx

        self.model = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=n_gpu_layers,
            n_batch=n_batch,
            n_ctx=n_ctx,
            f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
            verbose=True,
        )
        super().__init__()

    def send_request(self, X, break_word:str = " name:") -> str:
        tok_len = self.model.get_num_tokens(X)
        print(tok_len)
        if tok_len > 1500:
            raise Exception(f"Request exceeds prefelable 1500 tokens. Has: {tok_len}")
        prev = ""
        res = ""
        for token in self.model.stream(X, echo=False):
            res += token
            if break_word == prev+token:
                # print(res)#dev
                res = res.replace(" ; name:", "")
                res = res.replace(";name:", "")
                res = res.replace(" ;name:", "")
                res = res.replace("; name:", "")
                res = res.replace(" name:", "")
                res = res.replace("name:", "")
                break
            prev = token
        
        res = res.replace("  ;  ", "")
        res = res.replace("  ; ", "")
        res = res.replace("  ;", "")
        res = res.replace(" ; ", "")
        res = res.replace("; ", "")
        res = res.replace(" ;", "")
        res = res.replace(";", "")
        return res

    def get_info(self) -> str:
        return {
            "model":"LLama-13b-hf-q8_0",
            "model_path":self.model_path,
            "n_gpu_layers":self.n_gpu_layers,
            "n_batch":self.n_batch,
            "n_ctx":self.n_ctx,
        }


In [3]:
# test libs

from pprint import pprint
import pandas as pd
import numpy as np
from transformers import LlamaTokenizerFast

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# test request

df = pd.read_csv("train.csv")

def compose(gdf):
    comp = [f"name: {e[2]} ; ingredients: {e[4]} ; preparation:{e[3]}" for e in gdf.values[:-1]]
    # print(gdf.values[-1])
    comp = comp + [f"name: {e[2]} ; ingredients: {e[4]} ; preparation:" for e in [gdf.values[-1]]]
    return " ; ".join(comp)

# req = compose(df.iloc[[11, 15, 112, 122, 133, 144, 155, 8]])
# req = "The second planet in the solar system is "

In [5]:
model = LlamaCPP()


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4060 Ti, compute capability 8.9, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from models/llama-13b-hf_q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length 

In [6]:
import random
import time

I_ITER = 100


reqs = []
ress = []

start = time.time()
for _ in range(I_ITER):
    req = compose(df.iloc[[random.randint(0, len(df)) for _ in range(8)]])
    res = model.send_request(req)
    reqs.append(req)
    ress.append(res)
    print("----------------------------------------------------------------------------------------------------------------------------------------")
    print(req)
    print(res)
end = time.time()
print((end - start)/I_ITER)


1281
----------------------------------------------------------------------------------------------------------------------------------------
name: Swabian noodles spatzle ; ingredients: flour, salt, egg, water ; preparation:Fill large kettle half-full of water and bring to a boil. Combine flour, salt and eggs in a medium-sized mixing bowl, stirring with a fork. Add water until batter stretches 6 - 7 inches before tearing when pulled up on a spoon. Place large spoonful of batter on wooden cutting board, tilt it over kettle and slice thin strips of batter off into the boiling water. Dip knife into cold water between slices to prevent batter from sticking. Cook for 1 - 2 minutes and remove from a slotted spoon. Drain and place in a warmed serving bowl. Continue until batter is gone. ; name: Fluffy strawberry jello dessert ; ingredients: strawberry jell-o gelatin dessert, boiling water, vanilla ice cream, ice cubes, whipped cream, strawberry ; preparation:Combine jello and boiling water i

Llama.generate: prefix-match hit


----------------------------------------------------------------------------------------------------------------------------------------
name: Meatloaf sensation ; ingredients: ground beef, salsa, taco seasoning, egg, breadcrumbs, mexican blend cheese, salt, pepper ; preparation:Combine all ingredients, except half of the taco seasoning. Mix well. Shape into loaf and place in slow cooker. Sprinkle with remaining taco seasoning. Cover. Cook on low 8-10 hours. ; name: Honeyed mustard chicken ; ingredients: chicken, butter, honey, prepared mustard, salt, curry powder ; preparation:Preheat oven to 375f. Melt butter in saucepan. Add honey, mustard, salt and curry. Pour mixture over chicken and bake for 45-60 min. In covered dish. Serve extra sauce over rice. Serves 4. ; name: Miss daisy s hot baked chicken salad ; ingredients: cooked chicken, celery, pecans, mayonnaise, salt, lemon juice, onions, cheddar cheese, potato chips ; preparation:Preheat oven to 350. Combine first 7 ingredients. Po

Llama.generate: prefix-match hit


----------------------------------------------------------------------------------------------------------------------------------------
name: Quiche with brie ; ingredients: pie crusts, brie cheese, cream cheese, butter, cream, eggs, chives, salt, white pepper, cayenne pepper ; preparation:Line a 9-inch tart pan with a removable bottom with the pie crust. Line the inside of the crust with crumbled wax paper or parchment paper and fill with dried beans. Place in oven and bake about 12-15 minutes or until partially done. Remove the beans and the paper and bake an additional 2-3 minutes to dry out the bottom. Preheat oven to 375f to bake the quiche. Using a fork, blend the cheese with the butter and cream. Beat in the eggs. Force this mixture through a sieve to remove any lumps. Stir in the chives. Taste and season with the salt, pepper and cayenne. Pour into the pastry, place the pan on a baking sheet and bake for 25 to 30 minutes or until the quiche is puffed and browned. ; name: Apple

...

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1035.75 ms
llama_print_timings:      sample time =      24.97 ms /   138 runs   (    0.18 ms per token,  5527.07 tokens per second)
llama_print_timings: prompt eval time =    1619.48 ms /  1132 tokens (    1.43 ms per token,   698.99 tokens per second)
llama_print_timings:        eval time =   10091.74 ms /   137 runs   (   73.66 ms per token,    13.58 tokens per second)
llama_print_timings:       total time =   11955.55 ms /  1269 tokens
Llama.generate: prefix-match hit


----------------------------------------------------------------------------------------------------------------------------------------
name: Thai inspired sweet potato fries ; ingredients: sweet potatoes, olive oil, garlic clove, sambal oelek chili paste, reduced sodium soy sauce ; preparation:Preheat oven to 425f. Combine oil, garlic, chili paste and soy sauce in a bowl. Add sweet potatoes and toss to coat. Spread single layer on baking sheet. Bake for 20 to 30 minutes, string once or twice to prevent burning and allow even cooking. ; name: Junk bars aka seven layer bars ; ingredients: fiber one cereal, water, unsweetened coconut, butterscotch chips, nuts, chocolate chips, fat-free sweetened condensed milk ; preparation:Preheat oven to 350f. Mix water and crumbs in a 13x9 pan. Using the bottom of a large measuring cup, press the moist crumbs down. Evenly distributing them using a smaller measuring cup, get the corners. Sprinkle coconut on top followed by butterscotch, nuts, and choc

Llama.generate: prefix-match hit


----------------------------------------------------------------------------------------------------------------------------------------
name: Weight watchers hot tuna melt on english muffin ; ingredients: tuna in water, celery, low-fat whipped cream cheese, salt, pepper, lemon juice, whole wheat english muffin, reduced-fat cheddar cheese ; preparation:Preheat broiler or toaster oven to high. Combine tuna, celery, cream cheese, salt, pepper, and lemon juice in a small bowl. Mix well until thoroughly combined. Scoop equal parts tuna mixture onto each muffin half. Sprinkle equal amounts cheese over each tuna-topped muffin. Place both halves in broiler or toaster oven and heat until cheese is melted, 1-2 minutes, watching carefully so they do not burn. Remove promptly from heat. ; name: Granny s simple way fried chicken ; ingredients: broiler-fryer chicken, salt, flour, oil ; preparation:Wash chicken. My granny always cuts the breast halves in half. Salt all chicken pieces good on both si

Exception: Request exceeds prefelable 1500 tokens. Has: 1502

In [9]:
df = pd.DataFrame()

In [12]:
ress

['Preheat oven to 400 degrees F. Press 1 biscuit into each of 6 ungreased muffin cups. Bake 5 minutes. In 12-inch skillet, saut chicken, garlic, and vegetables in wine until tender. Add broth; cover. Simmer 5 minutes. Stir in soup. Pour mixture into 6 cups; top each with biscuit. Bake 12 minutes or until biscuits are browned.',
 'Combine first 8 ingredients. Add milk and mix well. Add eggs and mix well. Combine crushed crackers and bacon and mix well. Add to meat mixture and mix well. Shape into 3 loaves and place in greased baking pan. Bake at 350 for 1 hour. Add 1/2 cup water to canned soup and mix well. Spread over top and bake for an additional 1/2 hour.',
 'In a punch bowl mix together the chocolate milk and coffee. Pour in the coffee liqueur and stir to combine. Add ice and stir to combine. Serve in tall glasses and garnish with an orange wedge.',
 'In a small bowl, combine sugar and cinnamon. Dip bread slices in water and then in sugar mixture. Place on baking sheet and bake at 